In [1]:
!pip install gym
!pip install stable_baselines3[extra]

In [1]:
from Pokemon import Pokemon, Pokemon_Move, Pokemon_Battle
import gym
from gym import Env
import numpy as np

Sceptile has 130 hp.
Blastoise has 139 hp.
Leaf-blade: 15 pp, 90 power, 100 accuracy, grass
Shadow-ball: 15 pp, 80 power, 100 accuracy, ghost
Swords-dance: 20 pp, None power, 100 accuracy, normal
Bullet-seed: 30 pp, 25 power, 100 accuracy, grass
Choose a move: leaf-blade
Super Effective!
2.0
Crunch: 15 pp, 80 power, 100 accuracy, dark
Leer: 30 pp, None power, 100 accuracy, normal
Surf: 15 pp, 90 power, 100 accuracy, water
Ice-beam: 10 pp, 90 power, 100 accuracy, ice
Choose a move: Ice-beam
Super Effective!
2.0
Sceptile has 46.8 hp.
Blastoise has 31.171428571428578 hp.
Leaf-blade: 15 pp, 90 power, 100 accuracy, grass
Shadow-ball: 15 pp, 80 power, 100 accuracy, ghost
Swords-dance: 20 pp, None power, 100 accuracy, normal
Bullet-seed: 30 pp, 25 power, 100 accuracy, grass
Choose a move: bullet-seed
Super Effective!
2.0
Blastoise has fainted.


In [5]:
 # Define the observation space for continuous attributes (Pokemon stats and move damage dealt)
num_stats = 6  # Number of Pokemon stats (HP, Attack, Defense, SpAttack, SpDefense, Speed)
num_moves = 4  # Number of moves each Pokemon can have
observation_stats_low = np.zeros(num_stats)
observation_stats_high = np.array([255.0] * num_stats)

observation_damage_low = np.zeros(num_moves)
observation_damage_high = np.array([100.0] * num_moves)

        # Define the observation space for discrete attributes (types, move types, and previously used moves)
num_types = 18  # Number of Pokemon types
num_damage_types = 3  # Number of damage types (special, physical, status)
observation_types_low = np.zeros(num_types)
observation_types_high = np.array([1.0] * num_types)
observation_damage_types_low = np.zeros(num_damage_types)
observation_damage_types_high = np.array([1.0] * num_damage_types)

        

In [4]:
class BattleEnv(Env):
    def __init__(self):
        #Four moves for Pokemon
        self.action_space = Discrete(4)
        #Combine the continuous and discrete attributes
        self.observation_space = self.observation_space = gym.spaces.Tuple([
            gym.spaces.Box(low=self.observation_stats_low, high=self.observation_stats_high, dtype=np.float32),
            gym.spaces.MultiDiscrete([num_types, num_types]),      # Player's Pokemon types
            gym.spaces.MultiDiscrete([num_types, num_types, num_types, num_types]),  # Move types of the player's Pokemon moves
            gym.spaces.Box(low=self.observation_damage_low, high=self.observation_damage_high, dtype=np.float32),  # Damage dealt by the player's Pokemon moves
            gym.spaces.MultiBinary(num_moves),                    # Previous moves used by the player's Pokemon
            gym.spaces.MultiDiscrete([num_types, num_types, num_types, num_types]),  # Move types of the opponent's Pokemon moves
            gym.spaces.MultiBinary(num_moves),                    # Previous moves used by the opponent's Pokemon
        ])
        
    def step(self, action):
        opponent_action = Discrete(4).sample()
        
        print(f'{self.Pokemon1.name.capitalize()} has {self.Pokemon1.hp} hp.')
        print(f'{self.Pokemon2.name.capitalize()} has {self.Pokemon2.hp} hp.')

        if self.Pokemon1.speed > self.Pokemon2.speed:
            self.perform_turn(self.Pokemon1, self.Pokemon2, action)
            if self.Pokemon2.hp > 0:  # Check if the battle has already ended
                self.perform_turn(self.Pokemon2, self.Pokemon1, opponent_action)
        elif self.Pokemon2.speed > self.Pokemon1.speed:
            self.perform_turn(self.Pokemon2, self.Pokemon1, opponent_action)
            if self.Pokemon1.hp > 0:
                self.perform_turn(self.Pokemon1, self.Pokemon2, action)
        else:
            first_pokemon = random.choice([self.Pokemon1, self.Pokemon2])
            second_pokemon = self.Pokemon2 if first_pokemon == self.Pokemon1 else self.Pokemon1
            self.perform_turn(first_pokemon, second_pokemon, action if first_pokemon == self.Pokemon1 else opponent_action)
            if second_pokemon.hp > 0:  # Check if the battle has already ended
            self.perform_turn(second_pokemon, first_pokemon)

        if self.Pokemon1.hp <= 0:
            print(f'{self.Pokemon1.name.capitalize()} has fainted.')
            self.done = True
        if self.Pokemon2.hp <= 0:
            print(f'{self.Pokemon2.name.capitalize()} has fainted.')
            self.done = True

        observation = self.get_observation()
        reward = self.get_reward()

            #return observation, reward, done, info
    def render(self):
        print(f'{self.Pokemon1.name.capitalize()} has {self.Pokemon1.hp} hp.')
        print(f'{self.Pokemon2.name.capitalize()} has {self.Pokemon2.hp} hp.')
    def reset(self):
        self.Pokemon1 = self.get_random_pokemon()  # Get a random player Pokemon
        self.Pokemon2 = self.get_random_pokemon()  # Get a random opponent Pokemon
        self.current_turn_number = 1
        self.any_status_condition_active = False
        self.previous_action = None
        self.previous_opponent_action = None

        # Get the initial observation
        observation = self.get_observation()

        return observation
    
    def get_reward(self):
        beg_health = self.Pokemon2.hp
        if self.done:
            if self.Pokemon1.hp <= 0:
                # Player Pokemon has fainted, so the agent lost the battle
                reward = -100  # Penalty for losing the battle
            elif self.Pokemon2.hp <= 0:
                # Opponent Pokemon has fainted, so the agent won the battle
                reward = 100  # Reward for winning the battle
            else:
                # The battle ended in a draw or some other unknown condition
                reward = 0
        else:
            reward = beg_health - self.Pokemon2.hp
        
        return reward
    
    def get_observation(self):
        player_observation = np.array([self.Pokemon1.hp, self.Pokemon1.attack, self.Pokemon1.defense,
                                       self.Pokemon1.spattack, self.Pokemon1.spdefense, self.Pokemon1.speed],
                                      dtype=np.float32)
        return player_observation
    
    def get_random_Pokemon(self):
        pass

TypeError: Pokemon_Battle.__init__() missing 2 required positional arguments: 'Pokemon1' and 'Pokemon2'